# Import libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [ ]:
import time
import pandas as pd
from tqdm import tqdm

# Set up Selenium


In [ ]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (no GUI)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Crawl Data

## Lấy danh sách links

lấy link và lưu vào dataframe

In [ ]:
df_link = pd.DataFrame(columns=['links'])

In [ ]:
pages = 200
progress_bar_pages = tqdm(total=pages, desc='Processing_pages')

for page in range(pages):
    page = str(page+1)
    url = "https://www.chotot.com/mua-ban-may-tinh-de-ban?page="+page
    driver.get(url)
    time.sleep(3)

    links = driver.find_elements(By.CLASS_NAME, "AdItem_adItem__gDDQT")
    length = len(links)
    progress_bar = tqdm(total=length, desc='Processing_'+page)
    for num in range(length):
        df_link.loc[len(df_link)] = links[num].get_attribute('href')
        progress_bar.update(1)
    progress_bar.close()
    progress_bar_pages.update(1)

progress_bar_pages.close()

Processing_pages: 100%|██████████| 200/200 [22:22<00:00,  6.71s/it]


lưu danh sách link vào file csv

In [ ]:
df_link.to_csv('./data_test/link_may_ban.csv',index=True)
print("done")

done


## Vào từng link, lấy dữ liệu về cấu hình máy và giá của từng máy

Lấy link từng sản phẩm

In [ ]:
frame_link = pd.read_csv('./data_test/link_may_ban.csv')

In [ ]:
df_data = pd.DataFrame(columns=['pc_brand', 'pc_model', 'elt_condition', 'elt_warranty', 'desktop_screen_size', 'pc_cpu',
                                'pc_ram', 'pc_vga', 'pc_drive_capacity', 'elt_origin', 'usage_information', 'classify', 'price'])

In [ ]:
df_data.to_csv('./data_test/data_may_ban.csv',index=False)
print("done")

done


In [ ]:
# length = 5
length = len(frame_link)
progress_bar = tqdm(total=length, desc='Processing')

for num_link in range(length):
    link = frame_link['links'][num_link]
    driver.get(link)
    time.sleep(3)
    df_data_link = pd.DataFrame(columns=['pc_brand', 'pc_model', 'elt_condition', 'elt_warranty', 'desktop_screen_size', 'pc_cpu',
                            'pc_ram', 'pc_vga', 'pc_drive_capacity', 'elt_origin', 'usage_information', 'classify', 'price'])
    arr_data = {}
    arr_data['classify'] = 'desktop'
    data = driver.find_elements(By.CLASS_NAME, "AdParam_adParamValue__IfaYa")
    if(data != []):
        for num_data in range(len(data)):
            arr_data[data[num_data].get_attribute("itemprop")] = data[num_data].text

        price = driver.find_elements(By.CLASS_NAME, "AdDecription_price__L2jjH")
        arr_data['price'] = price[0].text

    df_data_link.loc[len(df_data_link)]=arr_data
    df_data_link.to_csv('./data_test/data_may_ban.csv', mode='a', header=False, index=False)

    progress_bar.update(1)

progress_bar.close()


Processing:  65%|██████▌   | 2623/4010 [4:05:17<2:09:42,  5.61s/it]


In [ ]:
frame_data = pd.read_csv('./data_test/data_may_ban.csv')
frame_data

,pc_brand,pc_model,elt_condition,elt_warranty,desktop_screen_size,pc_cpu,pc_ram,pc_vga,pc_drive_capacity,elt_origin,usage_information,classify,price
0,NaN,NaN,Đã sử dụng (chưa sửa chữa),>12 tháng,>= 21 inch,Intel Xeon,> 32GB,NVIDIA,256 GB,Việt Nam,In trên bao bì,desktop,13.500.000 đ
1,NaN,NaN,Đã sử dụng (chưa sửa chữa),Còn bảo hành,>= 21 inch,Intel Core i5,8 GB,Khác,500 GB,Đang cập nhật,In trên bao bì,desktop,1.800.000 đ
2,NaN,NaN,Đã sử dụng (chưa sửa chữa),Còn bảo hành,>= 21 inch,Intel Xeon,32 GB,NVIDIA,256 GB,Đang cập nhật,In trên bao bì,desktop,8.500.000 đ
3,NaN,NaN,Mới,Còn bảo hành,>= 21 inch,Intel Xeon,> 32GB,NVIDIA,256 GB,Việt Nam,In trên bao bì,desktop,12.500.000 đ
4,NaN,NaN,Đã sử dụng (chưa sửa chữa),3 tháng,Không bán kèm màn hình,Intel Core i3,8 GB,NVIDIA,128 GB,Đang cập nhật,In trên bao bì,desktop,6.300.000 đ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,NaN,NaN,Mới,Còn bảo hành,>= 21 inch,Intel Core i3,8 GB,Onboard,250 GB,Đang cập nhật,In trên bao bì,desktop,10.500.000 đ
4006,NaN,NaN,Đã sử dụng (chưa sửa chữa),>12 tháng,>= 21 inch,Intel Core i3,16 GB,AMD,500 GB,Việt Nam,In trên bao bì,desktop,10.500.000 đ
4007,NaN,NaN,Đã sử dụng (chưa sửa chữa),Hết bảo hành,>= 21 inch,Intel Core i5,8 GB,Onboard,1 TB,Nhật Bản,In trên bao bì,desktop,6.500.000 đ
4008,NaN,NaN,Đã sử dụng (chưa sửa chữa),Bảo hành hãng,NaN,NaN,NaN,NaN,NaN,Đang cập nhật,In trên bao bì,desktop,3.000.000 đ
